### Import necessary libraries

In [1]:
import sys
import timeit

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import networkx as nx

from sklearn.metrics import accuracy_score, precision_score, recall_score

### Check if GPU is available

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

### Helper functions

In [3]:
def create_adjacency(g):
    A     = nx.to_numpy_matrix(g)
    A_hat = A/np.linalg.norm(A, ord=1, axis=1, keepdims=True)
    
    A_full = np.zeros((dim,dim))
    n = A_hat.shape[0]
    
    A_full[:n,:n] = A_hat
    
    return A_full

### Define a Neural Network class

In [4]:
class MyGCN(nn.Module):
    
    def __init__(self):
        super(MyGCN, self).__init__()
        self.W_gcn = nn.ModuleList([nn.Linear(dim, dim) for _ in range(layer)])
        self.W_property = nn.Linear(dim, 2)
    
    def update(self, xs, adjacency, i):
        hs = torch.relu(self.W_gcn[i](xs))
        return torch.matmul(adjacency, hs)
    
    def forward(self, g):
        adjacency = torch.FloatTensor(create_adjacency(g)).to(device)
        n = adjacency.size()[0]
        
        xs = torch.FloatTensor(np.eye(dim)).to(device)
        
        for i in range(layer):
            xs = self.update(xs, adjacency, i)
        
        xs = torch.unsqueeze(torch.sum(xs,0),0)
        z_properties = self.W_property(xs)
        
        return z_properties
        
    def __call__(self, index, train=True):
        if index<=44:
            t_properties = torch.LongTensor([1]).to(device)
            G = nx.read_gpickle('pos_' + str(int(index)) + '.gpickle')
        else:
            G = nx.read_gpickle('neg_' + str(int(index-44)) + '.gpickle')
            t_properties = torch.LongTensor([0]).to(device)
            
        z_properties = self.forward(G)
        
        if train:
            loss = F.cross_entropy(z_properties, t_properties)
            return loss
        else:
            zs     = torch.softmax(z_properties,1).to('cpu').data.numpy()
            ts     = t_properties.to('cpu').data.numpy()
            labels = np.argmax(zs)
            
            return labels, ts

### Create a trainer class

In [5]:
class Trainer(object):
    
    def __init__(self, model):
        self.model = model
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=0.01)
        
    def train(self, dataset_train):
        np.random.shuffle(dataset_train)
        N = len(dataset_train)
        loss_total = 0
        for i in range(0, N):
            data_batch = dataset_train[i]
            loss = self.model(data_batch)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_total += loss.to('cpu').data.numpy()
        return loss_total

### Create a tester class

In [6]:
class Tester(object):
    
    def __init__(self, model):
        self.model = model

    def test(self, dataset_test):

        N = len(dataset_test)
        label_list, t_list = [], []

        for i in range(0, N):
            data_batch = dataset_test[i]
            labels, ts = self.model(data_batch, train=False)
            label_list = np.append(label_list, labels)
            t_list     = np.append(t_list, ts)
        
        auc       = accuracy_score(t_list, label_list)
        precision = precision_score(t_list, label_list)
        recall    = recall_score(t_list, label_list)
        
        return auc, precision, recall
    
    def result(self, epoch, time, loss, auc_dev, auc_test, precision, recall, file_name):
        with open(file_name, 'a') as f:
            result = map(str, [epoch, time, loss, auc_dev, auc_test, precision, recall])
            f.write('\t'.join(result) + '\n')

### Model Hyperparameters

In [7]:
dim            = 171
layer          = 2
lr             = 1e-3
lr_decay       = 0.75
decay_interval = 20
iteration      = 200
num_examples   = 44+35

(dim, layer, decay_interval, iteration, num_examples) = map(int, [dim, layer, decay_interval, iteration, num_examples])
lr, lr_decay                            = map(float, [lr, lr_decay])

### Data preparation

In [8]:
def split_dataset(dataset, ratio):
    n = int(ratio * len(dataset))
    dataset_1, dataset_2 = dataset[:n], dataset[n:]
    return dataset_1, dataset_2

dataset = np.linspace(1,num_examples,num_examples)
np.random.shuffle(dataset)

dataset_train, dataset_   = split_dataset(dataset, 0.8)
dataset_dev, dataset_test = split_dataset(dataset_, 0.5)

### Create and train model

In [9]:
torch.manual_seed(1234)

model   = MyGCN().to(device)
trainer = Trainer(model)
tester  = Tester(model)

print('Training...')
print('Epoch \t Time(sec) \t Loss_train \t AUC_dev \t AUC_test \t Precision \t Recall')

start = timeit.default_timer()

setting = 'layers_' + str(layer)

file_result = 'output/' + setting + '.txt'
with open(file_result, 'w') as f:
    f.write('Epoch \t Time(sec) \t Loss_train \t AUC_dev \t AUC_test \t Precision_test \t Recall_test\n')

for epoch in range(iteration):
    if (epoch+1) % decay_interval == 0:
        trainer.optimizer.param_groups[0]['lr'] *= lr_decay

    loss    = trainer.train(dataset_train)
    auc_dev = tester.test(dataset_dev)[0]
    auc_test, precision, recall = tester.test(dataset_test)
    
    lr_rate = trainer.optimizer.param_groups[0]['lr']

    end  = timeit.default_timer()
    time = end - start

    tester.result(epoch, time, loss, auc_dev, auc_test, precision, recall, file_result)
    print('%d \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f \t %.4f' %(epoch, time, loss, auc_dev, auc_test, precision, recall))

Training...
Epoch 	 Time(sec) 	 Loss_train 	 AUC_dev 	 AUC_test 	 Precision 	 Recall


C:\Users\mayan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0 	 1.0582 	 73.5401 	 0.6250 	 0.6250 	 0.0000 	 0.0000
1 	 1.4922 	 50.5432 	 0.3750 	 0.3750 	 0.3750 	 1.0000
2 	 1.9056 	 50.5998 	 0.3750 	 0.3750 	 0.3750 	 1.0000
3 	 2.3349 	 45.6404 	 0.5000 	 0.5000 	 0.0000 	 0.0000
4 	 2.7572 	 50.7776 	 0.6250 	 0.6250 	 0.5000 	 0.6667
5 	 3.1919 	 43.7368 	 0.3750 	 0.3750 	 0.3750 	 1.0000
6 	 3.6107 	 51.3717 	 0.3750 	 0.3750 	 0.3750 	 1.0000
7 	 4.0159 	 47.3253 	 0.3750 	 0.3750 	 0.3750 	 1.0000
8 	 4.4174 	 38.9070 	 0.5000 	 0.5000 	 0.3333 	 0.3333
9 	 4.8541 	 43.6153 	 0.3750 	 0.3750 	 0.3750 	 1.0000
10 	 5.2577 	 40.7827 	 0.3750 	 0.3750 	 0.3750 	 1.0000
11 	 5.6828 	 39.1521 	 0.5000 	 0.5000 	 0.3333 	 0.3333
12 	 6.0820 	 42.5631 	 0.5000 	 0.3750 	 0.3333 	 0.6667
13 	 6.5013 	 36.9182 	 0.5000 	 0.5000 	 0.4000 	 0.6667
14 	 6.9391 	 33.1159 	 0.3750 	 0.3750 	 0.3750 	 1.0000
15 	 7.3651 	 35.1306 	 0.3750 	 0.3750 	 0.3750 	 1.0000
16 	 7.7972 	 43.0887 	 0.6250 	 0.5000 	 0.4000 	 0.6667
17 	 8.2210 	 35.5388 	 

140 	 60.0739 	 2.6796 	 0.3750 	 0.5000 	 0.4000 	 0.6667
141 	 60.4780 	 2.6734 	 0.3750 	 0.5000 	 0.4000 	 0.6667
142 	 60.8727 	 2.6676 	 0.3750 	 0.5000 	 0.4000 	 0.6667
143 	 61.2687 	 2.6483 	 0.5000 	 0.5000 	 0.4000 	 0.6667
144 	 61.7140 	 2.6413 	 0.3750 	 0.5000 	 0.4000 	 0.6667
145 	 62.1455 	 2.6045 	 0.5000 	 0.5000 	 0.4000 	 0.6667
146 	 62.5483 	 2.7030 	 0.3750 	 0.5000 	 0.4000 	 0.6667
147 	 62.9560 	 2.6474 	 0.5000 	 0.5000 	 0.4000 	 0.6667
148 	 63.3890 	 3.0872 	 0.5000 	 0.5000 	 0.4000 	 0.6667
149 	 63.7843 	 2.6238 	 0.3750 	 0.5000 	 0.4000 	 0.6667
150 	 64.1886 	 2.5654 	 0.5000 	 0.5000 	 0.4000 	 0.6667
151 	 64.5825 	 2.5173 	 0.5000 	 0.5000 	 0.4000 	 0.6667
152 	 64.9839 	 2.4538 	 0.5000 	 0.5000 	 0.4000 	 0.6667
153 	 65.4084 	 2.4706 	 0.5000 	 0.6250 	 0.5000 	 1.0000
154 	 65.8060 	 2.3658 	 0.5000 	 0.6250 	 0.5000 	 1.0000
155 	 66.2365 	 2.4992 	 0.3750 	 0.5000 	 0.4000 	 0.6667
156 	 66.6367 	 2.3120 	 0.3750 	 0.5000 	 0.4000 	 0.66